# RISE Camp Capstone Exercise

To finish up RISE Camp this year, we're going to build an end-to-end application that takes advantage of some of the projects we've learned about in the last couple days. Those of you who attended last year's Camp will remember the Pong exercise that trained an RL policy in Ray and deployed it in Clipper. Today's version of Pong is more feature rich: We will take advantage of Flor's experiment tracking and WAVE's encryption capabilities, in addition to leveraging a new Clipper deployment model and RLlib's policies.

As a brief overview, we will train three ML models in this exercise. The first two models will use [imitation learning](https://blog.statsbot.co/introduction-to-imitation-learning-32334c3b1e7a) to learn how to play Pong, and the third will train a reinforcement learning policy using RLlib and Ray. Flor will track the training processes for all three models. We will also encrypt each one of these models with WAVE and deploy & serve the models in Clipper.

For those of you unfamiliar with imitation learning, the approach is simple. We previously recorded the actions of some humans playing Pong. Using this training data, we will build a logistic regression classifier that selects a paddle action (up, down, or stay) based on the current location, velocity, and trajectory of the ball.

Finally, you'll play a game (or more!) against each of the three models. We'll aggregate the results to see which model performs the best.

In [ ]:
# Python compatibility imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import pong_py
import cloudpickle

# ray imports
import ray
from ray.tune.registry import register_env
from ray.rllib.agents import ppo

import flor

In [ ]:
# set Flor metadata for the notebook
flor.setNotebookName('integration.ipynb')

In a separate notebook, we have set up a WAVE client and defined some helper functions that we'll use below. Feel free to look at the `wave-setup.ipynb` file in this directory if you'd like to dig in.

In [ ]:
%run wave-setup.ipynb

In [ ]:
# call Wave helper function to create granting and receiving entities
orgEntity, recipientEntity = createWaveEntities()

# Imitation Learning

First, let's take a look at our data. Run the cells below to load and inspect one of the two imitation learning datasets we'll use below.

In [ ]:
import pandas as pd
df = pd.read_csv('imitation-small.csv')

print(df.columns)
df.head()

As you can see, we have many labeled actions (`up`, `down`, or `stop`), in addition to the corresponding state of the game -- the location of one user's paddle (we ignore the other user's actions), the location and velocity of the ball, and the previous location of the ball.

## Model Training

First, we're going to define three Flor functions:

* `preproc_imitation`: cleans the input data
* `train_imitation_model`: trains an imitation learning model on the input data
* `encrypt_model`: encrypts that model using Wave

The preprocessing function reads an input CSV and converts the `up`, `down`, and `stay` labels into numerical values. It also normalizes the all the numerical values (the location of the controlled paddle, the location & velocity of the ball, and the previous location of the ball).

In [ ]:
# PRE-DEFINED FLOR FUNCTION. CELL IS NOT EDITABLE.

@flor.func
def preproc_imitation(imitation_data, procd_imitation_data, **kwargs):
    import pandas as pd
    df_data = pd.read_csv(imitation_data)
    
    # drop the user column because we don't want to train on it
    df_data = df_data.drop(labels="user", axis=1)

    # discretize the labels
    def convert_label(label):
        """Convert labels into numeric values"""
        if(label=="down"):
            return 1
        elif(label=="up"):
            return 2
        else:
            return 0

    # apply the discretization function defined above
    df_data['label'] = df_data['label'].apply(convert_label)
    
    # normalize all the numerical values
    df_data.loc[:, "leftPaddle_y":"ball_y_prev"] = df_data.loc[:, "leftPaddle_y":"ball_y_prev"]/500.0
    
    # write out the data to a file
    df_data.to_json(procd_imitation_data)

The model training function takes a JSON blob of the cleaned data and fits a SciKit Learn logistic regression model to classify the action to take based on the input features. The model is pickled and dumped into a file.

In [ ]:
# PRE-DEFINED FLOR FUNCTION. CELL IS NOT EDITABLE.

@flor.func
def train_imitation_model(procd_imitation_data, model, **kwargs):
    import cloudpickle
    import pandas as pd
    from sklearn import linear_model
    
    # read in the data
    df_data = pd.read_json(procd_imitation_data)
    
    # separate the desired label from the rest of the data
    labels = df_data['label']
    training_data= df_data.drop(['label'], axis=1)

    # train the model
    skmodel = linear_model.LogisticRegression()
    skmodel.fit(training_data, labels)
    
    # write to a file
    with open(model, 'wb') as f:
        cloudpickle.dump(skmodel, f)

Finally, the `encrypt_model` function takes the model we trained above and a handle to a WAVE entity that has access to all models. It uses the WAVE entity to encrypt the model and serializes the ciphered model into a file.

In [ ]:
# PRE-DEFINED FLOR FUNCTION. CELL IS NOT EDITABLE.

@flor.func
def encrypt_model(granting_entity, model, model_tag, encrypted_model, **kwargs):
    import wave3 as wv
    granting_entity = deserializeEntity(granting_entity)
    
    # read the model binary, so we can encrypt it
    with open(model, 'rb') as f:
        model = f.read()
    
    # NOTE: We are relying on a global handle to WAVE here. 
    # In practice, we would have to recreate this handle explicitly.
    encrypt_response = wave.EncryptMessage(
        wv.EncryptMessageParams(
            # the namespace is the organization
            namespace=granting_entity.hash,
            resource="models/pong/" + model_tag,
            content=model))
    
    with open(encrypted_model, 'wb') as f:
        f.write(encrypt_response.ciphertext)

Next, we define a Flor experiment called `pong-imitation` and link together the input data and the functions defined above. 

In [ ]:
DATA_FILE = 'imitation-small.csv'

# get the small or large tag from the DATA_FILE variable
model_tag = DATA_FILE.split('.')[0].split('-')[1]

# where to serialize the deserialize the Wave entity
ENTITY_FILE = 'org_entity.bin'

with flor.Experiment('pong-imitation') as ex:
    # load data into an artifact
    imitation_data = ex.artifact(DATA_FILE, 'imitation_data')
    
    # call preprocessing function
    do_preproc_imitation = ex.action(preproc_imitation, [imitation_data])
    procd_imitation_data = ex.artifact('imitation_data.json', 'procd_imitation_data', do_preproc_imitation)
    
    # train the model 
    do_train_imitation_model = ex.action(train_imitation_model, [procd_imitation_data])
    model = ex.artifact('model.pkl', 'model', do_train_imitation_model)
    
    model_tag = ex.literal(name='model_tag', v=model_tag)
    
    # serialize the wave entity, so we can track it as an artifact
    serializeEntity(orgEntity, ENTITY_FILE)
    granting_entity = ex.artifact(ENTITY_FILE, 'granting_entity')
    
    do_encrypt_model = ex.action(encrypt_model, [granting_entity, model, model_tag])
    encrypted_model = ex.artifact('encrypted_model.bin', 'encrypted_model', do_encrypt_model)

In [ ]:
encrypted_model.plot()

Lastly, we will call `pull` on the encrypted model. As we saw in the Flor tutorial, this will run the whole pipeline and generate the model. The `utag` argument will be used to differentiate from different versions of the model we train.

In [ ]:
encrypted_model.pull(utag=model_tag.v)
model_location = encrypted_model.resolve_location()[0]

## Model Deployment

Now that we have a model that we've trained, we are going to deploy it in Clipper. Run the cell below to start Clipper on your EC@ instance.

In [ ]:
# NOTE: YOU SHOULD ONLY RUN THIS CELL ONCE!

import logging
import sys
import subprocess

from clipper_admin import DockerContainerManager, ClipperConnection
from clipper_admin.deployers import python as py_deployer
from clipper_util.auth_deployer import auth_deploy_python_model

# Make logging work correctly
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Start Clipper locally
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.stop_all()
clipper_conn.start_clipper()

And we will attempt to deploy that model. You'll notice that we've wrapped the standard Clipper API with a special authenticated deployer that only deploys the model if it can decrypt it with the given recipient entity.

Running the cell below _**should initially fail**_. This is because we haven't yet granted the `recipientEntity` permission to read the model that we've encrypted. Run the cell below to see for yourself.

Once we deploy the model, we will create a new application called `pong-{model_name}` and link the model to that application.

In [ ]:
def deploy_model(clipper_conn, tag, model_loc, model_version=1):
    model_name = "pong-policy-" + tag
    app_name = "pong-" + tag
    
    # load the encrypted model into memory
    with open(model_loc, 'rb') as f:
        ciphered_model = f.read()

    auth_deploy_python_model(clipper_conn, model_name,wave,
        recipientEntity, ciphered_model, version=1, input_type="doubles")

    clipper_conn.register_application(name=app_name, default_output="0", input_type="doubles", slo_micros=100000)
    clipper_conn.link_model_to_app(app_name=app_name, model_name=model_name)

deploy_model(clipper_conn, model_tag.v, model_location)

Now, we're going to go ahead and use the helper function defined above to grant permission to the recipient entity to decrypt the ciphered model. Run the cell below to grant decryption permission and deploy the model to Clipper.

In [ ]:
grantPermission(orgEntity, recipientEntity, 'models/pong/' + model_tag.v)
deploy_model(clipper_conn, model_tag.v, model_location, model_version=1)

Now, for comparison's sake, we're going to train another imitation learning model with a different dataset -- `imitation-large.csv`. This dataset incorporates training data from two different players, and it should ideally perform better than the previous version. 

Run the cell below to use Flor's update features to change the value of the `model_tag` literal and the imitation data artifacts. It will then recreate the encrypted model with the new data and grant permission to the WAVE entity to access it. Finally, it will deploy the model using the function above.

In [ ]:
# define a new data file
DATA_FILE = 'imitation-large.csv'

# update the imitation_data artifact to point at the new file
imitation_data.loc = DATA_FILE

# update the model tag literal based on this new fine
model_tag.v = DATA_FILE.split('.')[0].split('-')[1]

# train the model on the new data file2
encrypted_model.pull(utag=model_tag.v)
model_location = encrypted_model.resolve_location()[0]

# grant decryption permissions for the new model
grantPermission(orgEntity, recipientEntity, 'models/pong/' + model_tag.v)

# deploy the new model
deploy_model(clipper_conn, model_tag.v, model_location, model_version=1)

Run the cell below to start the pong server locally, and you'll be given a URL which you can open in your browser to play against the two pong models you've just trained. You'll select the models as opponents in the dropdown menu on the left hand side. You'll notice there's a reinforcement learning option there as well -- we'll come back to that model soon. For now, you should just play a game against the `Imitation Learning (Small)` and `Imitation Learning (Large)` models.

In [ ]:
clipper_addr = clipper_conn.get_query_addr()

import subprocess
server_handle = subprocess.Popen(["./start_webserver.sh", clipper_addr], stdout=subprocess.PIPE)
print(str(server_handle.stdout.readline().strip(), 'utf-8'))

You'll notice that, surprisingly, the model trained on the large dataset performs significantly worse than the model trained on the small dataset. Let's take a look at the difference between the two and see why:

In [ ]:
# read the two CSVs into dataframes
import pandas as pd
large = pd.read_csv('imitation-large.csv')
small = pd.read_csv('imitation-small.csv')

# complicated Pandas syntax to diff the two datasets
large[large.merge(small, on=list(large.columns), how='left', indicator=True)['_merge'] == 'left_only']

We see that all the entries in the large dataset have been made by Ion, who apparently isn't very good at Pong. As a result, the noise from these entries worsens the performance of the model in practice.

# Reinforcement Learning

Next, we'll train an RL policy to play pong. We will use Proximal Policy Optimization (PPO) to train this agent. We'll see that the RL policy trained here will perform significantly better than either of the imtiation learning models we trained above. As above, we'll first define a couple Flor functions. We will also reuse the encrypt model function defined above.

First, we need to either start Ray or make sure it's already running on our machine:

In [ ]:
@flor.func
def start_ray(**kwargs):
    try:
        ray.get([])
    except:
        ray.init()    
    return {'exit_code': 0}

Next, we define a function called `train_agent` that takes an environment configuration and trains the model for a specified number of iterations. Both the environment config and the number of iterations will be Flor literals.

In [ ]:
@flor.func
def train_agent(env_config, num_iterations, model, **kwargs):
    import cloudpickle as cp
    
    # create an RL lib environment and register it
    register_env("pong_env", lambda ec: pong_py.PongJSEnv())
    agent = ppo.PPOAgent(env="pong_env", config={"env_config": {}})

    # train for the specified number of iterations
    for i in range(num_iterations):
        result = agent.train()
        
    # save the model to a file
    with open(model, 'wb') as f:
        f.write(agent.save_to_object())

Finally, we'll string all of these functions together into another Flor experiment called `rl-pong`.

In [ ]:
model_tag = 'rl'

with flor.Experiment('rl-pong') as ex:
    # make sure that Ray is running before attempting to train a model
    do_start_ray = ex.action(start_ray, [])
    exit_code = ex.literal(name='exit_code', parent=do_start_ray)
    
    # define configurations variables relevant to training the RL model
    env_config = ex.literal({}, 'env_config') # TODO: Fill env_config
    num_iterations = ex.literal(50, 'num_iterations')

    # setup the training action and the save location of the checkpoint
    do_train_agent = ex.action(train_agent, [env_config, num_iterations, exit_code])
    model = ex.artifact('model.pkl', 'model', do_train_agent)
    
    model_tag = ex.literal(name='model_tag', v=model_tag)
    
    # serialize the wave entity, so we can track it as an artifact
    serializeEntity(orgEntity, ENTITY_FILE)
    granting_entity = ex.artifact(ENTITY_FILE, 'granting_entity')
    
    do_encrypt_model = ex.action(encrypt_model, [granting_entity, model, model_tag])
    encrypted_model = ex.artifact('encrypted_model.bin', 'encrypted_model', do_encrypt_model)

We'll plot the new experiment graph:

In [ ]:
encrypted_model.plot()

And we'll run this pipeline in the usual fashion. Because we're training for 50 iterations, this might take a few minutes to run to completion.

In [ ]:
encrypted_model.pull()
model_location = encrypted_model.resolve_location()[0]

Once again, we have to grant permission to the entity to decrypt the model, but this time, we'll use a wildcard, letting it decrypt anything in the `models/pong/` resource space.

In [ ]:
grantPermission(orgEntity, recipientEntity, 'models/pong/*')

And deploy the model to Clipper under a new application name (`pong-rl`). Note that this code is the same as the cell above but is repeated for convenience.

In [ ]:
from clipper_util.auth_deployer import auth_deploy_rllib_model

# load the encrypted model into memory
with open(model_location, 'rb') as f:
    ciphered_model = f.read()

model_name = "pong-policy-" + model_tag.v
app_name = "pong-" + model_tag.v

def predict(model, states):
    return [str(model.compute_action(s)) for s in states]

auth_deploy_rllib_model(
    clipper_conn,
    model_name,
    predict,
    wave,
    recipientEntity,
    ciphered_model,    
    version=1,
    input_type="doubles"
)

clipper_conn.register_application(name=app_name, default_output="0", input_type="doubles", slo_micros=100000)
clipper_conn.link_model_to_app(app_name=app_name, model_name=model_name)

Now, we can go back to the Pong tab and play against the RL agent we just deployed. We expected the RL agent to be much better than either of the imitation learning models, but the difference might not be discernible in a single game. We're aggregating the reuslts from all the players in the room to see which model has the best point differential. The graph on the projector is showing `points_scored - points_conceded` for each of the three models.

In this exercise, we:

* trained 3 ML models: two using imitation learning, and one using reinforcement learning via Ray and RLlib 
* captured every one of these model training pipeline using Flor
* used Flor's tracking to diff the datasets and understand why one performed better than the other. 
* encrypted each model with WAVE and deployed them into Clipper

We hope you had a great RISE Camp and look forward to seeing you again next year!